<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/JuliaTensorRQI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" />
$\newcommand{\R}{\mathbb{R}}$
# Colab Notebook For Generalized Rayleigh Quotient Iteration and Rayleigh Chebyshev Iteration For Tensor Eigenvalue with a homogeneous constraint.

* Solving the problem $A(I, X^{[m-1]}) = \lambda B(I, X^{[d-1]})$ for a tensor $A$ of degree $m$ and $B$ of degree $d$, $X\in \R^n$. The constraint is $B(X)=1$, $B$ is a tensor of degree $d$.
* Use the library Arblib for higher numerical precision to demonstrate cubic convergence.

# Result: Confirming RQI has quadratic and Rayleigh-Chebyshev has cubic convergence order.
* A sample numerical simulation:
For random tensors $A$ and $B$ with $n=6, m=3, d=2$ at warm up point with error $\lvert A(I, x) - \lambda B(I, x)\rvert = 7.250e-03$ The Newton and Chebyshev sequence of residual errors are

|Iteration | Rayleigh(Newton) | Rayleigh Chebyshev (Chebyshev)|
|---|---|---|
|1|2.594e-04|2.042e-05|
|2|3.080e-07|8.671e-14|
|3|2.509e-13|2.690e-39|
|4|1.049e-25|5.503e-115|


# You can view the results as-is. If you want to run the code follow the instructions below
## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). **This takes a couple of minutes.**
4. **Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.**

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-02-13 19:08:52 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
**REMEMBER TO LOAD THE PAGE BY RUNNING F5 IF the following command does not work**

The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, znver2)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# Use Arblib to test the package to show higher precision

Which demostrates Chebyshev has cubic convergence and Newton has quadratic convergence

In [2]:
using Pkg
Pkg.add("Arblib")
Pkg.add("PyCall")

using Arblib
using LinearAlgebra
using Printf
import Random

prec = 512
# limitting to use m, d <= 3 so we can use Arblib
using PyCall



    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed IrrationalConstants ─ v0.1.1
   Installed OpenBLAS32_jll ────── v0.3.17+0
   Installed FLINT_jll ─────────── v200.900.4+0
   Installed Arb_jll ───────────── v200.2300.0+0
   Installed Arblib ────────────── v0.8.1
   Installed LogExpFunctions ───── v0.3.21
   Installed SpecialFunctions ──── v2.1.7
   Installed Compat ────────────── v4.6.0
   Installed ChainRulesCore ────── v1.15.7
   Installed OpenSpecFun_jll ───── v0.5.5+0
   Installed InverseFunctions ──── v0.1.8
   Installed ChangesOfVariables ── v0.1.5
   Installed DocStringExtensions ─ v0.9.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [fb37089c] + Arblib v0.8.1
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [fb37089c] + Arblib v0.8.1
  [d360d2e6] + ChainRulesCore v1.15.7
  [9e997f8a] + ChangesOfVariables v0.1.5
  [34da2185] + Compat v4.6.0
  [ffbed154] + DocStringExtensions v0.9.3
  [3587e190] + Inverse

# a few functions to generate random symmetric tensors

In [3]:
py"""
import numpy as np
def generate_symmetric_tensor(k, m, seed):
    np.random.seed(seed)
    A = np.full(tuple(m*[k]), np.nan)
    current_idx = np.zeros(m, dtype=int)
    active_i = m - 1
    A[tuple(current_idx)] = np.random.rand()
    while True:
        if current_idx[active_i] < k - 1:
            current_idx[active_i] += 1
            if np.isnan(A[tuple(current_idx)]):
                i_s = tuple(sorted(current_idx))
                if np.isnan(A[i_s]):
                    A[i_s] = np.random.rand()
                A[tuple(current_idx)] = A[i_s]
        elif active_i == 0:
            break
        else:
            next_pos = np.where(current_idx[:active_i] < k-1)[0]
            if next_pos.shape[0] == 0:
                break
            current_idx[next_pos[-1]] += 1
            current_idx[next_pos[-1]+1:] = 0
                        
            active_i = m - 1
            if np.isnan(A[tuple(current_idx)]):
                i_s = tuple(sorted(current_idx))
                if np.isnan(A[i_s]):
                    A[i_s] = np.random.rand()
                A[tuple(current_idx)] = A[i_s]
    return A
"""

function RandSymmetricArb3Tensor(k, seed)
    Apy = py"generate_symmetric_tensor"(k, 3, seed)
    A = Vector{ArbMatrix}(undef, k)
    for i in 1:k
        A[i] = ArbMatrix(Apy[:, :, i], prec=prec)
    end
    return A, Apy
end    

function RandSymmetricArbMatrix(k, seed)
    B = py"generate_symmetric_tensor"(k, 2, seed)
    return ArbMatrix(B, prec=prec)
end

@inline function RandArbVec(k)
    return ArbVector(Random.rand(k), prec=prec)
end    

function randAB(m, d, k, seed)
    if m == d
        seed_d = seed + 5
    else
        seed_d = seed
    end
    if m == 2
        A = RandSymmetricArbMatrix(k, seed)
    else    
        A, Apy = RandSymmetricArb3Tensor(k, seed)    
    end
    
    if d == 2
        B = RandSymmetricArbMatrix(k, seed_d)
    else
        B, Bpy = RandSymmetricArb3Tensor(k, seed_d)
    end
    return A, B
end    


randAB (generic function with 1 method)

# Helper functions to evaluate tensors

In [4]:

function EvalTensor(T, x, m)
    # function, tensor, gradient. Relation:
    # Fv' = m*F1, Fv^(2) = m(m-1)F2, Fv^(3) = (m-2)(m-1)m F3
    # F1' = (m-1)*F2
    # F2' = (m-2)*F3
    # m is 2 or 3 to work with ArbMatrix
    if m == 2
        F2 = 0
        F1 = T
        F0 = T*x
        Fv = sum(F0.*x)
    else
        F2 = T
        F1 = ArbMatrix(zeros(k, k), prec=prec)
        for i in 1:k
            F1 .+= x[i]*F2[i]
        end
        F0 = F1*x
        Fv = sum(F0.*x)
    end
    return Fv, F0, F1, F2
end    



EvalTensor (generic function with 1 method)

Helper function for the constraint set $M$ defined by $B(x) = 1$:
* Generating a random point
* Generating random vector at the random point
* Projection *proj_tan* to the tangent bundle $TM$
* Projection $\Pi$ to the vector bundle $E_{\Pi}$
* Retraction *Trtr* by scaling down to $B(x) = 1$ if $B(x) > 0$.
* $get\_xperp$ gives the complement basis of x $assume $x^Tx =1$

In [5]:
function randPoint()
    while true
        x = RandArbVec(k)
        vv = EvalTensor(B, x, d)[1]
        if vv > 0
            return x / real_root(vv, d)
        end
    end
end    

function proj_tan(x, omg)
    # projection to the tangent bundle
    V = EvalTensor(B, x, d)
    return omg - x*sum(V[2].*omg)
end    

function Pi(x, omg)
    # projection to the vector bundle E
    V = EvalTensor(B, x, d)
    return omg - V[2]*sum(x.*omg)
end    

function randvec(x)
    return proj_tan(x, RandArbVec(k))
end    

function real_root(v, d)
    if (v < 0) & (d % 2 == 0)
        return NaN
    end
    return sign(v)*abs(v)^(1/d)
end        

function Trtr(x, eta)
    # retraction
    v = x+eta
    V = EvalTensor(B, v, d)
    return v / real_root(V[1], d)
end    

function get_xperp(x)
    Q = 2.0*(x[1] > 0) - 1
    P = x[1]/Q
    return vcat(
        -Q*reshape(x[2:end], 1, k-1),
        I(k-1)-1/(1+P)*reshape(x[2:end], k-1, 1)*reshape(x[2:end], 1, k-1))
end    


get_xperp (generic function with 1 method)

Assume there are tensors $A$ of size $k$ order $m$ and $B$ of size $k, d$, define the functions $L$ and its derivatives, function $Ray$ (its Rayleigh quotient) and function $F= L(x, Ray(x))$

In [6]:
function Ray(x)
    VA = EvalTensor(A, x, m)
    return sum(x.*VA[2])
end    


function L(x, lbd)
    VA = EvalTensor(A, x, m)
    VB = EvalTensor(B, x, d)
    Lv = VA[2] - VB[2]*lbd
    Lx = (m-1)*VA[3] - (d-1)*lbd*VB[3]
    Llbd = - VB[2]
    Lxx = Vector{ArbMatrix}(undef, k)
    
    for i in 1:k
        if (m == 2) & (d == 2)
            Lxx[i] = 0
        elseif m == 2
            Lxx[i] = ArbMatrix(- (d-2)*(d-1)*VB[4][i]*lbd, prec=prec)
        elseif d == 2
            Lxx[i] = ArbMatrix((m-2)*(m-1)*VA[4][i], prec=prec)
        else
            Lxx[i] = ArbMatrix((m-2)*(m-1)*VA[4][i] - (d-2)*(d-1)*VB[4][i]*lbd, prec=prec)
        end
    end
    Lxlbd = -(d-1)*VB[3]

    return Lv, Lx, Llbd, Lxx, Lxlbd
end

function F(x)
    VA = EvalTensor(A, x, m)
    VB = EvalTensor(B, x, d)
    lbd = sum(x.*VA[2])
    Lv = VA[2] - VB[2]*lbd
    Lx = (m-1)*VA[3] - (d-1)*lbd*VB[3]
    Llbd = - VB[2]
    Lxx = Vector{ArbMatrix}(undef, k)
    
    for i in 1:k
        if (m == 2) & (d == 2)
            Lxx[i] = 0
        elseif m == 2
            Lxx[i] = ArbMatrix(- (d-2)*(d-1)*VB[4][i]*lbd, prec=prec)
        elseif d == 2
            Lxx[i] = ArbMatrix((m-2)*(m-1)*VA[4][i], prec=prec)
        else
            Lxx[i] = ArbMatrix((m-2)*(m-1)*VA[4][i] - (d-2)*(d-1)*VB[4][i]*lbd, prec=prec)
        end
    end
    Lxlbd = -(d-1)*VB[3]
    
    return Lv, lbd, Lx, Llbd, Lxx, Lxlbd
end    


F (generic function with 1 method)

# Newton and Chebyshev steps
Assume global tensors are defined (to be defined in a moment

In [7]:
        
function NewtonInc(x)
    VB = EvalTensor(B, x, d)
    FF = F(x)
    tperp = get_xperp(VB[2]/norm(VB[2]))
    xperp = get_xperp(x/norm(x))
    P = I(k) - reshape(VB[2], k, 1)*reshape(x, 1, k)
    luLx = lu(xperp'*P*FF[3]*tperp)
    nts = ArbMatrix(zeros(k-1, 1), prec=prec)
    ldiv!(nts, luLx, xperp'*FF[1])
    return - tperp*nts
end    

function NewtonStep(x)
    VB = EvalTensor(B, x, d)
    FF = F(x)
    tperp = get_xperp(VB[2]/norm(VB[2]))
    xperp = get_xperp(x/norm(x))
    
    P = I(k) - reshape(VB[2], k, 1)*reshape(x, 1, k)
    luLx = lu(xperp'*P*FF[3]*tperp)
    nts = ArbMatrix(zeros(k-1, 1), prec=prec)
    ldiv!(nts, luLx, xperp'*FF[1])

    return Trtr(x, - tperp*nts)
end    

function ChevStep(x)
    VB = EvalTensor(B, x, d)
    FF = F(x)        
    tperp = get_xperp(VB[2]/norm(VB[2]))
    xperp = get_xperp(x/norm(x))

    P = I(k) - reshape(VB[2], k, 1)*reshape(x, 1, k)
    luLx = lu(xperp'*P*FF[3]*tperp)
    nts = ArbMatrix(zeros(k-1, 1), prec=prec)
    ldiv!(nts, luLx, xperp'*FF[1])
    
    ninc = - tperp*nts

    if (m == 2) & (d == 2)
        return Trtr(x, ninc)
    else
        Lxx2 = ArbMatrix(zeros(k, k), prec=prec)
        for i in 1:k
            Lxx2 .+= FF[5][i]*ninc[i]
        end
        PG = Pi(x, Lxx2*ninc)
        # np.tensordot(np.tensordot(FF[4], ninc, axes=1), ninc, axes=1)
        tts = ArbMatrix(zeros(k-1, 1), prec=prec)    
        ldiv!(tts, luLx, xperp'*PG)
        ginc = tperp*tts
        return Trtr(x, ninc - 0.5*ginc)
    end    
end

function NewtonSchur(x)
    VA = EvalTensor(A, x, m)
    VB = EvalTensor(B, x, d)
    FF = F(x)
    lbd = FF[2]
    Lx = (m-1)*VA[3] - (d-1)*lbd*VB[3]
    # luLx = lu(lx
    # LxiL = simple_solve(luLx, FF[0])
    # LxiLbd = la.solve((m-1)*VA[2] - (d-1)*VB[2]*lbd, FF[3])
    ret = ldiv!(lu(Lx), hcat(FF[1], FF[4]))
    eta = - ret[:, 1] + ret[:, 2]/sum(VB[2].*ret[:, 2])*sum(VB[2].*ret[:, 1])
    return Trtr(x, eta)
end


function ChevSchur(x)
    VA = EvalTensor(A, x, m)    
    VB = EvalTensor(B, x, d)
    FF = F(x)        
    lbd = FF[2]
    Lx = (m-1)*VA[3] - (d-1)*lbd*VB[3]
    luLx = lu(Lx)
    ret = ldiv!(luLx, hcat(FF[1], FF[4]))

    ninc = - ret[:, 1] + ret[:, 2]/sum(VB[2].*ret[:, 2])*sum(VB[2].*ret[:, 1])
    if (m == 2) & (d == 2)
        return Trtr(x, ninc)
    else
        Lxx2 = ArbMatrix(zeros(k, k), prec=prec)
        for i in 1:k
            Lxx2 .+= FF[5][i]*ninc[i]
        end
        ginc0 = ldiv!(luLx, Lxx2)*ninc
        ginc = ginc0 - ret[:, 2]/sum(VB[2].*ret[:, 2])*sum(VB[2].*ginc0)        
        return Trtr(x, ninc - 0.5*ginc)
    end    
end



ChevSchur (generic function with 1 method)

# Generating the tensors - check the Rayleigh and the Chebyshev steps are close in the Newton and Schur form

In [8]:
seed = 3
Random.seed!(seed)
k = 8
m = 3
d = 3
A, B = randAB(m, d, k, seed)

x = randPoint()

@printf("Newton diff %.3e\n", sum(NewtonStep(x) - NewtonSchur(x))/k^2)
@printf("Chev diff %.3e\n", sum(ChevStep(x) - ChevSchur(x))/k^2)


Newton diff 3.962e-19
Chev diff 7.047e-20


# Show convergence

In [9]:
function run_a_group(x)
  @printf("k=%d m=%d d=%d\n", k, m, d)
  for i in 1:30
      x = NewtonStep(x)
      err = norm(F(x)[1])
      @printf("WARM UP %.3e\n", err)
      if isnan(err) | (err < 5e-2)
          break
      end
  end

  xx = copy(x)
  for i in 1:30
      # xx = NewtonStep(xx)
      # eta = NewtonSchur(xx)
      xx = NewtonSchur(xx)
      err = norm(F(xx)[1])
      @printf("Newton %.3e\n", err)
      if isnan(err) | (err < 1e-24 )
          break
      end
  end

  xx = copy(x)
  for i in 1:30
      # xx = ChevStep(xx)
      xx = ChevSchur(xx)
      err = norm(F(xx)[1])
      @printf("Chebyshev %.3e\n", err)
      if isnan(err) | (err < 1e-40)
          break
      end
  end    
end
run_a_group(x)


k=8 m=3 d=3
WARM UP 1.928e+00
WARM UP 1.525e+00
WARM UP 8.686e-01
WARM UP 1.447e+00
WARM UP 1.382e+00
WARM UP 6.899e-01
WARM UP 1.398e-01
WARM UP 8.673e-02
WARM UP 8.301e-03
Newton 5.915e-04
Newton 1.290e-06
Newton 8.983e-12
Newton 8.546e-23
Newton NaN
Chebyshev 6.623e-05
Chebyshev 4.124e-12
Chebyshev 5.261e-34
Chebyshev NaN


# More tests:

In [10]:
seed = 2
Random.seed!(seed)
k = 4
m = 3
d = 2
A, B = randAB(m, d, k, seed)

x = randPoint()
run_a_group(x)

k=4 m=3 d=2
WARM UP 2.488e+00
WARM UP 1.674e-01
WARM UP 2.544e-02
Newton 1.060e-03
Newton 1.336e-06
Newton 3.608e-13
Newton 6.078e-26
Chebyshev 8.235e-05
Chebyshev 5.893e-14
Chebyshev 3.303e-40
Chebyshev 6.369e-119


In [11]:
seed = 0
Random.seed!(seed)
k = 6
m = 3
d = 2
A, B = randAB(m, d, k, seed)

x = randPoint()
run_a_group(x)

k=6 m=3 d=2
WARM UP 7.245e-02
WARM UP 7.250e-03
Newton 2.594e-04
Newton 3.080e-07
Newton 2.509e-13
Newton 1.049e-25
Chebyshev 2.042e-05
Chebyshev 8.671e-14
Chebyshev 2.690e-39
Chebyshev 5.503e-115
